In [1]:
import numpy as np

# Universal Soil Loss Equation

Written by: Matthew Yuen 

The Universal Soil Loss Equation (USLE) predicts the long-term average annual rate of erosion on a field slope based on rainfall pattern, soil type, topography, crop system, and management practices. USLE only predicts the amount of soil loss that results from sheet or rill erosion on a single slope and does not account for additional soil losses that might occur from gully, wind or tillage erosion. This erosion model was created for use in selected cropping and management systems, but is also applicable to non-agricultural conditions such as construction sites. A Revised Universal Soil Loss Equation (RUSLE) was created to address more detailed factors in the same equation. 

The Equation: 

$$ A = (R)(K)(LS)(C)(P) $$ 

where: 

A is the potential long-term average annual soil loss in tonnes per hectare per year.

R is the rainfall and runoff factor by geographic location.

K is the soil erodibility factor. IT is the measure of susceptibility of soil particles to detachment and transport by rain and runoff. 

LS is the length and slope steepness factor. 

C is the crop and vegetation management factor. 

P is the support practice factor. 

In [2]:
def A(R,K,LS,C,P):
    # R,K,LS,C,P are inputs
    return(R*K*LS*C*P)

## R Factor

R-factor is the long-term average rainfall conditions for specific geographic areas. 

We can determine the R factor by using figures presented in Wall et. al. 

    a) Eastern Canada
    - Locate the area of interest in Figure R-1 and Figure R-2.
    - Extrapolate point or area relative to R factor contours. 
    
    b) Prairie Provinces
    - Locate area of interest in Figures R-3a and R-3b
    - Add values to determine parameter R
    
    c) British Columbia
    - Locate area of interest in Figure R-4
    - Can convert to SI metric units by multiplying by 17.02

We can also calcualte seasonal R values:

    - Determine time of interest (eg. cropstage, season, months, etc.)
    - Select monthly distribution from climatic station closest to area of interest. 
    - Add the monthly values by the annual R for the time of interest
    - Multiply the value of total annual R value

Example of calculating R values: 

Using Figure R-4 by Wall et. al, we can find that the R-value for Vancouver, British Columbia is 850.

Suppose we wanted the seasonal R value from January to May. Adding up the values we get 44%. Then $0.44*850 = 374$. Thus the R value for Vancouver from January to May is 374. 

**What is the difference between the R-factor in USLE and RUSLE?** 

The R-factor is generally the same between USLE and RUSLE. 

## K Factor 

The K-factor is based on soil texture, organic matter content, permeability, and other factors inherent to soil type (Renard et. al). 
Calculation is based on five parameters: percentage of silt plus very fine sand (0.05 to 0.1 mm), percentage sand greater than 0.1 mm, organic matter content, structure, permeability. 

$$ 100K=2.1M^{1.14}(10^{-4})(12-a)+(3.25)(b-2)+(2.5)(c-3) $$

where:

M = (percent silt+very fine sand)x(100 - percent clay)

a = percent organic matter

b = soil structure code used in soil classification

c = profile permeability class

A table of K estimations based on information obtained on approximately 1600 samples collected in Southern Ontario by Ontario Institute of Pedology surveyors is shown by Wall et. al. 

In [3]:
def k(M,a,b,c):
    # M = (% silt + % very fine sand)*(100-%clay)
    # a is the percentage of organic matter
    # b is the structure code class of soil classification
    # c is permeability class
    return(1/100*(2.1*(M**1.14)*(10**(-4))*(12-a)+3.25*(b-2)+2.5*(c-3)))

Determining K Factors: 
1. Use the equation above if there is information on: 
    - percent sand, very fine sand and clay
    - percent organic matter
    - structure of soil
    - permeability

2. Use nomogrpah (Figure K-1, Part 2 in Wall et. al)
    - to obtain a K factor based on all parameters in method 1 or 
    - to approximiate a K factor, based on particle size percentages and organic matter

Example calculating K Factor: 

The attributes of a particular loam are as follows: 30% sand (25% very fine sand, 5% other sand diameters), 40% silt, 30% clay, 2.8% organic matter, fine granular structure, slow to moderate permeability. 

1. Using nomograph in Figure K-1 in Wall et. al, 
K = 0.04

2. If particle size distribution of the same soil was unknown but we could determine that the soil is a loam and probably contains more than 2% organic matter, then we can use Table K-3 to determine that we have fine sandy loam with organic material > 2%, K = 0.038. 
Likewise, we could be given the percentage of sand, silt, and clay and used Figure K-2 to find that this sample is clay loam with over 2% organic matter and find in Table K-3 that K = 0.037. 

3. Using the equation above: 

M = 4550, a = 2.8, b = 2, c = 4. 

Then K = 0.310851

In [4]:
k(4550,2.8,2,4)

0.31085126223336146

**What is the difference between K-factor in USLE and RUSLE?** 

The K-factor remains the same between USLE and RUSLE but also accounts for seasonal changes such as freezing, thawing, soil moisture, and soil consolidation in RUSLE (Renard et. al). 

## LS Factor

LS Factor accounts for effect of slope angle and length on erosion. It also adjusts the erosion prediction for a given slope length and slope angle to account for differences from conditions present at standard erosion monitoring plots. USLE was based on 22m long, 9% slopes. 

$$ LS = (0.065+0.0456(x)+0.006541(x)^2)(\frac{y}{c})^{NN} $$

where 

x = slope steepness in %

y = slope length in m

c = constant, 22.1 metric

NN = 0.2 if x < 1, 0.3 if 1$\leq$x<3, 0.4 if 3$\leq$x<5, else 0.5

For typical slope conditions, a 10% error in slope length results in a 5% error in computed soil loss (Renard et. al). Soil loss is much more sensitive to changes in slope steepness than errors in slope length. For example, in USLE, a 10% error in slope steepness will usually give about a 20% error in computer soil loss. RUSLE has a more linear slope steepness relationship than did the USLE. Improvements in the relationship for steep slopes mean that computed soil loss for slopes less than 20% are smilar in RUSLE and USLE, but on steep slopes, computed soil loss in RUSLE is just over half that predicted by USLE, whose relationship did not include data for steep slopes (Renard et. al). In addition, RUSLE makes more explicit the reliance of the length relationship on the susceptibility of the soil to rilling, which may be influenced by the slope steepness, soil characteristics, and management impacts.

In [5]:
def ls(x,y,NN):
    # x is the slope steepness in % 
    # y is the slope length in m 
    # NN is 0.2 if x<1, 0.3 if 1<x<3, 0.4 if 3<x<5, 0.5 else
    return((0.065+0.0456*x+0.006541*x**2)*(y/22.1)**NN)

The Revised Universal Soil Loss Equation has a different equation for calculating LS. 

$$L = \left(\frac{\lambda}{22.1}\right)^m$$ where L measures the slope length in meters, $\lambda$ is the horizontal projection and m can be calculated by $m = \frac{\beta}{1+\beta}$, where $\beta$ is related to the ratio of rill erosion caused by overland flow to interril erosion. 

In [6]:
def L(llambda,m):
    # llambda is the horizontal projection
    return((llambda/22.1)**m)

In [7]:
def m(beta):
    return (beta/(1+beta))

$$\beta = \left(\frac{\frac{\sin\theta}{0.0896}}{3(sin\theta)^{0.8}+0.56}\right)$$, with $\theta$ being the slope angle.

In [8]:
def beta(theta):
    # theta is the slope angle
    return((np.sin(theta)/0.0896)/(3*np.sin(theta)**(0.8)+0.56))

When soil is highly susceptible to rilling, it is recommended that $\beta$ be calculated as $2\beta.$ 

When conditions favour more interrill and less rill erosion, m should be decreased by $\frac{\beta}{2}$. 

$$ S = 10.8\sin\theta+0.03 \space \text{if the slope is less than 9%} $$

$$ S = 16.8\sin\theta-0.5 \space \text{if the slope is greater than 9%} $$

Slopes less than 4.6m in length should use the following formula: 

$$ S = 3(\sin\theta)^{0.8}+0.56$$

When managing freshly tilled thawing soil: 

$$ S = 10.8\sin\theta + 0.03 \space \text{if slope if less than 9%} $$

$$ S = \left(\frac{\sin\theta}{0.0896}\right)^{0.6} \space \text{if slope is greater than 9%} $$ 

Convex slopes will have a greater effective LS factor (ie. high erosion estimate) than a uniform slope with the same average gradient. Conversely, a concave slope will generally have a lower effective erosion rate than a uniform slope of the same average gradient. 

Soil loss increases more with steepness than with slope length. 

## C Factor

C Factor is used to determine the relative effectiveness of soil and crop management systems in terms of preventing or reducing soil loss. 

When using the USLE, C Factor is determined by selecting crop type and tillage method that corresponds to field and multiplying these two factors together. 

A revised USLE equation has a different computation for C Factor: 

$$ C = (SLR)*(EI)$$
where

SLR = soil loss ratio (ratio of soil loss under specific practice during crop stage to soil loss on a similar field of bare soil)

EI = erosivity index 

The SLR Factor takes into account the relative and compounded impact of crop canopy cover, crop residues, tillage practices and residual soil quality for a particular crop. SLRs will vary throughout the year and for different cropstages. SLR factors have been development for most crops and are integrated into the C Factors for the regions of Canada. 

EI factor is the proportion of annual erosive rainfall occurring during each cropstage. Generally, the greatest portion of erosive rains fall during the middle of the growing season. 

**What is the difference between C Factor in USLE and C Factor in a Revised USLE?**

C Values were reevaluated and revised in RUSLE to reflect additional information on the effects of cover and management on erosion. The outcome of these calculations however remain the same. The RUSLE SLR (ratio comparing soil losses under specific practices to bare soil conditions) is still determined for a specific period during the year. In USLE calculations, this period corresponded to variable-lenght cropstage periods, reflecting a crop's development. RUSLE divides the year into 15 day periods. 

## P Factor

By definition the P Factor is the ratio of soil loss with a specific support practice to the corresponding loss with up and down slope cultivation and planning. In the absence of any support practice P, assumes unity and equals to 1 in USLE.

The lower the P value, the more effectively the practice helps to cause deposition to occur close to the source. 

General P Values: 

No support practice = 1 

Cross slope farming = 0.75

Contour farming (3-8% slopes) = 0.5

Strip cropping, cross slope (3-8% slopes) = 0.38 

Strip cropping, on contour (3-8% slopes) = 0.25

**What is the difference between the P-factor in USLE and RUSLE?** 

In USLE, the P-factor is based on installation practices that slow runoff and reduce soil movement. P-factor values change according to slope ranges with distinction for various ridge heights. In RUSLE, P-values are based on hydrologic soil groups, slope, row grade, ridge height, and 10-year single storm erosion index value. RUSLE computes the effect of stripcropping based on the transport capacity of flow in dense strips relative to the amount of sediment reaching the strip (Renard et. al). 

## Soil Loss Tolerance Rates

Very Low Soil Erosion: A < 6.7 hectare/year (3 tons/acre/year)

Low Soil Erosion: 6.7 hectare/year (3 tons/acre/year) < A < 11.2 hectare/year (5 tons/acre/year)

Moderate Soil Erosion: 11.2 hectare/year (5) < A < 22.4 hectare/year (10)

High Soil Erosion: 22.4 hectare/year (10) < A < 22.6 hectare/year (15) 

Severe Soil Erosion: A > 33.6 hectare/year (15)

## Example 1: 



Example from [Ministry of Agriculture, Food and Rural Affairs, "Universal Soil Loss Equation (USLE)", 2015. Accessed 21-Feb-2019.](http://www.omafra.gov.on.ca/english/engineer/facts/12-051.htm) 

Sample Field is Middlesex County, contains fine sandy loam soil with average organic matter content. The sample field is 244m long with 6% slope. The sample field was plowed in the spring and grain corn was planted. Cross-slope farming was used. 

R = 100

K = 0.4

LS = $(0.065+0.0456(6)+0.006541(6)^2)(\frac{244}{22.1})^{0.5}$ = 1.90752

C = $0.4*0.9 = 0.36 $

P = 0.75

In [9]:
A(100,0.4,1.90752,0.36,0.75)

20.601216

## Example 2: 

Using Vancouver at the Sample, contains loamy sand with average organic matter content, sample field is 300m long with 5% slope, growing the crop corn for grain with a conventional till, assume unity. 

R = $850/17.02 = 49.9412456$

K = 0.005 by Table K-3 in Wall et. al

LS = $(0.065+0.0456(5)+0.006541(5)^2)(\frac{300}{22.1})^{0.5} = 1.68201$

C = 0.31

P = 1

In [10]:
A(49.9412456,0.005,1.68201,0.31,1)

0.1302025954930668

## Bibliography

[Ministry of Agriculture, Food and Rural Affairs, "Universal Soil Loss Equation (USLE)", 2015. Accessed 21-Feb-2019.](http://www.omafra.gov.on.ca/english/engineer/facts/12-051.htm)

[Wall G.J., D.R. Coote, E.A. Pringle and I.J. Shelton (editors). 2002. RUSLEFAC - Revised Universal Soil Loss Equation for Application in Canada: A Handbook for Estimating Soil Loss from Water Erosion in Canada. Research Brand, Agriculture and Agri-Food Canada. Ottawa. Contribution No. AAFC/AAC2244E.117 pp.](http://sis.agr.gc.ca/cansis/publications/manuals/2002-92/rusle-can.pdf)

[Renard K.G., Yoder D.C., Lightle D.T., Dabney S.M., "Universal Soil Loss Equation and Revised Universal Soil Loss Equation", Handbook of Erosion Modelling 1st ed. pp. 137-167, 2011.](https://www.tucson.ars.ag.gov/unit/publications/PDFfiles/2122.pdf)

[Renard K.G., Foster G.R., Yoder D.C., McCool D.K., "RUSLE revisited: Status, questions, answers, and the future," J. Soil Water Conserv. 49(3):213-220. 1994.](http://cnrfiles.uwsp.edu/turyk/Database/Development/MJB/private/Thesis/JournalArticles/rusle.pdf)